In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Build a circuit to estimate the process error of. In this case we choose a $ZZ$ gate.

In [16]:
import cirq

qubits = cirq.LineQubit.range(2)
process_circuit = cirq.Circuit(cirq.ZZ(*qubits))

Define a noise model to simulate. Here we assume that single and two qubit gates are independently
affected by depolarising channels with some given probabilities $e_1$ and $e_2$.

In [17]:
class IndependentDepolariseNoiseModel(cirq.NoiseModel):
    """Applies single and two qubit depolarising channels independently"""

    def __init__(self, single_qubit_error: float, two_qubit_error: float) -> None:
        """Args:
        single_qubit_error: Single qubit pauli error
        two_qubit_error: Two qubit pauli error
        """
        super().__init__()
        self.single_qubit_error = single_qubit_error
        self.two_qubit_error = two_qubit_error

        self.single_qubit_depolarise = cirq.DepolarizingChannel(
            p=3 / 4 * single_qubit_error, n_qubits=1
        )
        self.two_qubit_depolarise = cirq.DepolarizingChannel(
            p=15 / 16 * two_qubit_error, n_qubits=2
        )

    def noisy_operation(self, operation: cirq.Operation) -> list[cirq.OP_TREE]:
        """Produces a list of operations by applying each noise model
        to the provided operation depending on the number of qubits it acts on.
        """
        if len(operation.qubits) == 1:
            return [operation, self.single_qubit_depolarise(*operation.qubits)]

        if len(operation.qubits) == 2:
            return [operation, self.two_qubit_depolarise(*operation.qubits)]

        return [operation]


noise = IndependentDepolariseNoiseModel(single_qubit_error=0.0005, two_qubit_error=0.02)

sim = cirq.DensityMatrixSimulator(
    noise=noise,
)

Now run the experiment.

In [18]:
import supermarq

experiment = supermarq.qcvv.CB(process_circuit, num_channels=4)
experiment.prepare_experiment(num_circuits=50, cycle_depths=[2, 8])
experiment.run_with_simulator(simulator=sim, repetitions=100)
if experiment.collect_data():
    result = experiment.analyze_results()

Building circuits:   0%|          | 0/400 [00:00<?, ?it/s]

Simulating circuits:   0%|          | 0/400 [00:00<?, ?it/s]

We now compare the results to the analytic process error. Note that although the process circuit
only contains a two qubit gate, the Cycle Benchmarking protocol estimates the error of the dressed
circuit. In our case this adds two single qubit gates to each cycle. Therefore we expect the
overall process error to be $e_2 + 2 e_1$

In [19]:
print(f"Estimated process error: {1-result.process_fidelity}")
print(f"Analytic_process error: {noise.two_qubit_error + 2 * noise.single_qubit_error}")

Estimated process error: 0.02066662697287236
Analytic_process error: 0.021


In [20]:
experiment.raw_data

,pauli_channel,cycle_depth,expectation,circuit,00,01,10,11
0,YI,4,0.94,process,0.94,0.03,0.00,0.03
1,YI,4,0.94,process,0.02,0.95,0.01,0.02
2,YI,4,0.96,process,0.04,0.94,0.01,0.01
3,YI,4,0.92,process,0.02,0.94,0.01,0.03
4,YI,4,0.96,process,0.03,0.95,0.01,0.01
...,...,...,...,...,...,...,...,...
395,ZX,16,0.72,process,0.07,0.13,0.73,0.07
396,ZX,16,0.62,process,0.10,0.04,0.77,0.09
397,ZX,16,0.76,process,0.04,0.81,0.07,0.08
398,ZX,16,0.72,process,0.81,0.09,0.05,0.05
